# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# install needed libraries
!pip install pyarrow
!pip install fastparquet
!pip install geopandas
!pip install pytest
!pip install keplergl

In [2]:
# all import statements needed for the project, for example:

import math

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db
import sqlite3
import numpy as np
import matplotlib.animation as animation
import keplergl
from keplergl import KeplerGl
import statistics
import unittest

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [4]:
# Calculate the distance between the two coordinates
def calculate_distance(from_coord: list, to_coord: list) -> float:
    R = 6373.0
    lat1 = math.radians(from_coord[0])
    lon1 = math.radians(from_coord[1])
    lat2 = math.radians(to_coord[0])
    lon2 = math.radians(to_coord[1])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

In [5]:
# For the dataset that is not given the trip distance, calculate the distance using the given coordinate data and add it to the dataframe
def add_distance_column(dataframe: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    distance = []
    for index, row in dataframe.iterrows():
        distance.append(calculate_distance((row['pickup_latitude'], row['pickup_longitude']), (row['dropoff_latitude'], row['dropoff_longitude'])))
    dataframe['trip_distance'] = distance
    return dataframe

### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [6]:
# get taxi taxi data from the website and generate a list of urls
def find_taxi_parquet_urls() -> list:
    response = requests.get(TAXI_URL)
    soup = bs4.BeautifulSoup(response.text, "html.parser")
    a = soup.find_all(lambda tag:'title' in tag.attrs and tag.attrs['title'] == "Yellow Taxi Trip Records")
    hrefs = [link.get('href') for link in a]
    hrefs_09 = list(filter(lambda href: '2009' in href, hrefs))
    hrefs_10 = list(filter(lambda href: '2010' in href, hrefs))
    hrefs_11 = list(filter(lambda href: '2011' in href, hrefs))
    hrefs_12 = list(filter(lambda href: '2012' in href, hrefs))
    hrefs_13 = list(filter(lambda href: '2013' in href, hrefs))
    hrefs_14 = list(filter(lambda href: '2014' in href, hrefs))
    hrefs_15 = list(filter(lambda href: '2015' in href, hrefs))
    for i in range(6):
        hrefs_15.pop()
    hrefs_t = hrefs_09 + hrefs_10 + hrefs_11 + hrefs_12 + hrefs_13 + hrefs_14 + hrefs_15
    return hrefs_t

In [25]:
find_taxi_parquet_urls()

['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-05.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-06.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-07.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-08.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-09.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-10.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-11.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-12.pa

In [26]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2012-01.parquet')

In [27]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2012-01-01 00:07:56,2012-01-01 00:12:09,1,0.9,1,N,158,231,2,4.9,0.5,0.5,0.00,0.0,0.0,5.90,None,None
1,1,2012-01-01 00:18:49,2012-01-01 00:30:01,1,2.3,1,N,231,164,2,8.5,0.5,0.5,0.00,0.0,0.0,9.50,None,None
2,1,2012-01-01 00:31:38,2012-01-01 00:46:05,1,2.2,1,N,164,148,2,9.3,0.5,0.5,0.00,0.0,0.0,10.30,None,None
3,1,2012-01-01 00:47:35,2012-01-01 00:55:57,4,0.9,1,N,148,107,2,5.3,0.5,0.5,0.00,0.0,0.0,6.30,None,None
4,1,2012-01-01 00:57:08,2012-01-01 01:02:42,3,0.7,1,N,107,107,2,4.5,0.5,0.5,0.00,0.0,0.0,5.50,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13058343,1,2012-01-31 23:43:30,2012-01-31 23:57:10,1,3.4,1,N,162,231,1,10.9,0.5,0.5,1.00,0.0,0.0,12.90,None,None
13058344,1,2012-01-31 23:04:26,2012-01-31 23:23:21,1,4.3,1,N,148,50,2,14.5,0.5,0.5,0.00,0.0,0.0,15.50,None,None
13058345,1,2012-01-31 23:58:54,2012-02-01 00:05:33,1,2.0,1,N,237,75,1,6.9,0.5,0.5,1.58,0.0,0.0,9.48,None,None
13058346,1,2012-01-31 23:10:40,2012-01-31 23:19:57,1,3.4,1,N,13,68,1,10.1,0.5,0.5,3.00,0.0,0.0,14.10,None,None


In [28]:
# Define a function that converts location to coordinates, and generate a dataframe
def convert_id_to_coord(df: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    shapefile = gpd.read_file(r"C:\Users\Silvia\Downloads\taxi_zones.shp")
    # Convert the geometry column in the shapefile into specific coordinates of latitude and longitude
    shapefile = shapefile.to_crs(4326)
    shapefile['latitude'] = shapefile['geometry'].centroid.y
    shapefile['longitude'] = shapefile['geometry'].centroid.x
    
    df = df
    df = df.loc[df["pulocationid"] <= 263]
    df = df.loc[df["pulocationid"] != 0]
    df = df.loc[df["dolocationid"] <= 263]
    df = df.loc[df["dolocationid"] != 0]
    # convert location IDs into longitude and latitude
    PUlongitude = []
    PUlatitude = []
    DOlongitude = []
    DOlatitude = []
    # convert the pickup location IDs into longitude and latitude
    for i in df['pulocationid']:
        PUlatitude.append(shapefile['latitude'][i-1])
        PUlongitude.append(shapefile['longitude'][i-1])
    for i in df['dolocationid']:
        DOlatitude.append(shapefile['latitude'][i-1])
        DOlongitude.append(shapefile['longitude'][i-1])
        
    df['pickup_longitude'] = PUlongitude
    df['pickup_latitude'] = PUlatitude
    df['dropoff_longitude'] = DOlongitude
    df['dropoff_latitude'] = DOlatitude
    # convert the drop off location IDs into longitude and latitude
    
    return df

In [29]:
# Obtain taxi data and clean the data
def get_and_clean_month_taxi_data(url: str) -> pd.core.frame.DataFrame:

    df = pd.read_parquet(url)
    df.columns = df.columns.str.lower()
    df_taxi = pd.DataFrame()

    # keep necessary columns into a new dataframe
    if 'tpep_pickup_datetime' in df.columns:
        df=df.rename(columns = {'tpep_pickup_datetime':'pickup_datetime',
                                'tip_amount' : 'tip_amount'})
        df=convert_id_to_coord(df)
        
    elif 'trip_pickup_datetime' in df.columns:
        df=df.rename(columns = {'trip_pickup_datetime':'pickup_datetime', 
                                'start_lon': 'pickup_longitude',
                                'start_lat': 'pickup_latitude',
                                'end_lon': 'dropoff_longitude',
                                'end_lat': 'dropoff_latitude',
                                'tip_amt' : 'tip_amount'})
        
    df.drop(df.columns.difference(['pickup_datetime',
                                    'trip_distance', 
                                    'pickup_latitude', 
                                    'pickup_longitude', 
                                    'dropoff_latitude', 
                                    'dropoff_longitude',
                                   'tip_amount']), 1, inplace=True)
    
    df=df[df["pickup_longitude"] <= -73.717047]  
    df=df[df["pickup_longitude"] >= -74.242330]
    df=df[df["pickup_latitude"] >= 40.560445]
    df=df[df["pickup_latitude"] <= 40.908524]
    df=df[df["dropoff_longitude"] <= -73.717047]
    df=df[df["dropoff_longitude"] >= -74.242330]
    df=df[df["dropoff_latitude"] >= 40.560445]
    df=df[df["dropoff_latitude"] <= 40.908524]

    df = df.loc[df["pickup_datetime"] != 0.0]
    
    return df

### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [30]:
# load uber data and clean the data
def load_and_clean_uber_data(csv_file: str) -> pd.core.frame.DataFrame:
    df = pd.read_csv(csv_file, on_bad_lines='skip')
    df.columns = df.columns.str.lower()
    add_distance_column(df)
    df.drop(df.columns.difference(['pickup_datetime',
                                     'trip_distance', 
                                     'pickup_latitude', 
                                     'pickup_longitude', 
                                     'dropoff_latitude', 
                                     'dropoff_longitude']), 1, inplace=True)

    # remove rows start and/or end outside of the following latitude/longitude coordinate box: 
    # (40.560445, -74.242330) and (40.908524, -73.717047)
    df=df[df["pickup_longitude"] <= -73.717047]  
    df=df[df["pickup_longitude"] >= -74.242330]
    df=df[df["pickup_latitude"] >= 40.560445]
    df=df[df["pickup_latitude"] <= 40.908524]
    df=df[df["dropoff_longitude"] <= -73.717047]
    df=df[df["dropoff_longitude"] >= -74.242330]
    df=df[df["dropoff_latitude"] >= 40.560445]
    df=df[df["dropoff_latitude"] <= 40.908524]

    # remove invalid rows thtat pickup time is 0
    df = df.loc[df["pickup_datetime"] != 0.0]

    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

    return df

Sampling the taxi data according to the number of uber trips each month

In [13]:
# Get the number of uber data per month
def get_number_to_sample() -> int:
    uber = load_and_clean_uber_data(r"C:\Users\Silvia\Downloads\uber_rides_sample.csv")
    uber.index = pd.to_datetime(uber['pickup_datetime'])
    number_each_month = uber.groupby(by=[uber.index.year, uber.index.month]).size()
    number = []
    for i in range(2009,2015):
        for j in range(1,13):
            number.append(number_each_month[i][j])
    for i in range(1,7):
        number.append(number_each_month[2015][i])
    return number

In [14]:
number = get_number_to_sample()

C:\Users\Silvia\AppData\Local\Temp\ipykernel_15864\1263730049.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['pickup_datetime',


In [16]:
number

[2504,
 2261,
 2665,
 2531,
 2619,
 2495,
 2454,
 2386,
 2448,
 2606,
 2479,
 2629,
 2618,
 1909,
 2338,
 2690,
 2739,
 2544,
 2536,
 2204,
 2422,
 2573,
 2455,
 2466,
 2346,
 2511,
 2768,
 2607,
 2681,
 2738,
 2515,
 2340,
 2577,
 2797,
 2623,
 2599,
 2592,
 2642,
 2822,
 2740,
 2722,
 2524,
 2516,
 2548,
 2611,
 2617,
 2525,
 2672,
 2598,
 2417,
 2739,
 2677,
 2547,
 2558,
 2455,
 2237,
 2547,
 2668,
 2556,
 2497,
 2451,
 2398,
 2697,
 2646,
 2600,
 2411,
 2285,
 2139,
 2340,
 2609,
 2351,
 2331,
 2188,
 2169,
 2324,
 2302,
 2417,
 2114]

In [17]:
def get_sample_taxi_data() -> pd.core.frame.DataFrame:
    sample_taxi_dataframes = []
    
    all_parquet_urls = find_taxi_parquet_urls()
    # do the sampling of taxi data according to the number of uber trip each month
    for i in range(0, 20): 
        n = number[i]
        url = all_parquet_urls[i]
        df = get_and_clean_month_taxi_data(url)
        df_sample = df.sample(n)
        sample_taxi_dataframes.append(df_sample)

    taxi_data = pd.concat(sample_taxi_dataframes)
    taxi_data['pickup_datetime'] = pd.to_datetime(taxi_data['pickup_datetime'])
    taxi_data = taxi_data.reset_index(drop = True)
    return taxi_data

In [18]:
sample20 = get_sample_taxi_data()

C:\Users\Silvia\AppData\Local\Temp\ipykernel_15864\2223629645.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['pickup_datetime',
C:\Users\Silvia\AppData\Local\Temp\ipykernel_15864\2223629645.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['pickup_datetime',
C:\Users\Silvia\AppData\Local\Temp\ipykernel_15864\2223629645.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['pickup_datetime',
C:\Users\Silvia\AppData\Local\Temp\ipykernel_15864\2223629645.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['pickup

In [24]:
sample20.to_csv('sample20.csv')  

In [43]:
def get_sample_taxi_data78() -> pd.core.frame.DataFrame:
    sample_taxi_dataframes = []
    
    all_parquet_urls = find_taxi_parquet_urls()
    # do the sampling of taxi data according to the number of uber trip each month
    for i in range(70, 78): 
        n = number[i]
        url = all_parquet_urls[i]
        df = get_and_clean_month_taxi_data(url)
        df_sample = df.sample(n)
        sample_taxi_dataframes.append(df_sample)

    taxi_data = pd.concat(sample_taxi_dataframes)
    taxi_data['pickup_datetime'] = pd.to_datetime(taxi_data['pickup_datetime'])
    taxi_data = taxi_data.reset_index(drop = True)
    return taxi_data

In [44]:
sample78 = get_sample_taxi_data78()

C:\Users\Silvia\AppData\Local\Temp\ipykernel_15864\4068850014.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shapefile['latitude'] = shapefile['geometry'].centroid.y
C:\Users\Silvia\AppData\Local\Temp\ipykernel_15864\4068850014.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shapefile['longitude'] = shapefile['geometry'].centroid.x
C:\Users\Silvia\AppData\Local\Temp\ipykernel_15864\2223629645.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['pickup_datetime',
C:\Users\Silvia\AppData\Local\Temp\ipykernel_15864\4068850014.py:6: UserWarning: Geometry is in a geographic CRS. Results 

In [45]:
sample78.to_csv('sample78.csv')  

In [40]:
def get_sample_taxi_data70() -> pd.core.frame.DataFrame:
    sample_taxi_dataframes = []
    
    all_parquet_urls = find_taxi_parquet_urls()
    # do the sampling of taxi data according to the number of uber trip each month
    for i in range(60, 70): 
        n = number[i]
        url = all_parquet_urls[i]
        df = get_and_clean_month_taxi_data(url)
        df_sample = df.sample(n)
        sample_taxi_dataframes.append(df_sample)

    taxi_data = pd.concat(sample_taxi_dataframes)
    taxi_data['pickup_datetime'] = pd.to_datetime(taxi_data['pickup_datetime'])
    taxi_data = taxi_data.reset_index(drop = True)
    return taxi_data

In [41]:
sample70 = get_sample_taxi_data70()

C:\Users\Silvia\AppData\Local\Temp\ipykernel_15864\4068850014.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shapefile['latitude'] = shapefile['geometry'].centroid.y
C:\Users\Silvia\AppData\Local\Temp\ipykernel_15864\4068850014.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shapefile['longitude'] = shapefile['geometry'].centroid.x
C:\Users\Silvia\AppData\Local\Temp\ipykernel_15864\2223629645.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['pickup_datetime',
C:\Users\Silvia\AppData\Local\Temp\ipykernel_15864\4068850014.py:6: UserWarning: Geometry is in a geographic CRS. Results 

C:\Users\Silvia\AppData\Local\Temp\ipykernel_15864\2223629645.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['pickup_datetime',


In [42]:
sample70.to_csv('sample70.csv')  

In [46]:
sample20

,pickup_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_amount
0,2009-01-10 13:23:16,2.000,-73.982541,40.763843,-73.987940,40.741227,1.92
1,2009-01-14 17:39:16,1.300,-73.995395,40.749472,-74.004407,40.742506,3.00
2,2009-01-26 06:21:00,19.260,-73.776892,40.645983,-73.982210,40.772557,0.00
3,2009-01-07 07:56:59,15.400,-73.863361,40.769973,-74.008209,40.703672,0.00
4,2009-01-15 17:54:00,0.171,-73.985552,40.747653,-73.966978,40.768760,0.00
...,...,...,...,...,...,...,...
49650,2010-08-23 19:03:56,3.500,-73.991474,40.750012,-73.953978,40.772962,0.00
49651,2010-08-01 17:27:36,1.700,-73.990118,40.734483,-73.974515,40.753011,0.00
49652,2010-08-23 04:34:02,1.400,-73.983996,40.725399,-73.996840,40.712136,1.12
49653,2010-08-08 19:19:00,4.610,-73.968080,40.802467,-73.989368,40.756822,0.00


In [47]:
allsample = pd.concat([sample20, sample40, sample50, sample60, sample70, sample78])

In [48]:
allsample

,pickup_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_amount
0,2009-01-10 13:23:16,2.000,-73.982541,40.763843,-73.987940,40.741227,1.92
1,2009-01-14 17:39:16,1.300,-73.995395,40.749472,-74.004407,40.742506,3.00
2,2009-01-26 06:21:00,19.260,-73.776892,40.645983,-73.982210,40.772557,0.00
3,2009-01-07 07:56:59,15.400,-73.863361,40.769973,-74.008209,40.703672,0.00
4,2009-01-15 17:54:00,0.171,-73.985552,40.747653,-73.966978,40.768760,0.00
...,...,...,...,...,...,...,...
18191,2015-06-01 12:25:22,2.880,-73.977698,40.758028,-74.008984,40.735035,0.00
18192,2015-06-07 23:02:31,12.100,-73.786533,40.646985,-73.917711,40.700522,7.46
18193,2015-06-18 07:41:01,0.840,-73.984196,40.759818,-73.977698,40.758028,0.00
18194,2015-06-09 12:59:25,0.500,-73.977698,40.758028,-73.970443,40.749914,1.95


In [49]:
allsample.to_csv('taxi_sample.csv')  

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def clean_month_weather_data_hourly(csv_file: str) -> pd.core.frame.DataFrame:
    # read file
    df = pd.read_csv(csv_file)
    #drop unnecessary colums
    df.drop(df.columns.difference(['DATE',
                                   'HourlyPrecipitation', 
                                   'HourlyWindSpeed']), 1, inplace=True)
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].replace('T', 0.0)
    # drop na values
    df.dropna(subset=['HourlyWindSpeed'], inplace=True)
    # convert "DATE" to datetime type
    df['DATE'] = pd.to_datetime(df['DATE'])
    # convert "HourlyPrecipitation" to float type
    df['HourlyPrecipitation'] = pd.to_numeric(df['HourlyPrecipitation'], errors='coerce')
    # fill in missing values
    df['HourlyPrecipitation'].fillna(0, inplace=True)
    # cast "df" to specified type
    df = df.astype({'HourlyWindSpeed': 'float32', 'HourlyPrecipitation': 'float32'})

    return df

In [ ]:
def clean_month_weather_data_daily(csv_file: str) -> pd.core.frame.DataFrame:
    # read file
    df = pd.read_csv(csv_file)
    # Replace data of the string type
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].replace('T', 0.0)
    # convert "DATE" to datetime type
    df['DATE'] = pd.to_datetime(df['DATE'])
    # convert "HourlyPrecipitation" to numeric type
    df['HourlyPrecipitation'] = pd.to_numeric(df['HourlyPrecipitation'], errors='coerce')
    # convert value of 'na' into 0.0
    df['HourlyPrecipitation'].fillna(0, inplace=True)
    #drop unnecessary colums
    df.drop(df.columns.difference(['DATE',
                                   'HourlyPrecipitation', 
                                   'HourlyWindSpeed']), 1, inplace=True)
    # calculate hourly average as a daily values
    df['DATE'] = df['DATE'].dt.date
    df = df.groupby('DATE', as_index=False).agg({'HourlyWindSpeed': np.mean, 'HourlyPrecipitation': np.mean})
    df['HourlyWindSpeed'] = df['HourlyWindSpeed'].map(lambda x: round(x, 2))
    # remame columns
    df.rename(columns={'HourlyWindSpeed': 'DailyAverageWindSpeed', 'HourlyPrecipitation': 'DailyPrecipitation'}, inplace=True)
    df = df.astype({'DailyAverageWindSpeed':'float32', 'DailyPrecipitation':'float32', 'DATE' : 'datetime64[ns]'})

    return df

In [ ]:
def load_and_clean_weather_data() -> pd.core.frame.DataFrame:
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = [
            "/content/2009_weather.csv",
            "/content/2010_weather.csv",
            "/content/2011_weather.csv",
            "/content/2012_weather.csv",
            "/content/2013_weather.csv",
            "/content/2014_weather.csv",
            "/content/2015_weather.csv"
        ]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [ ]:
def clean_sunset_sunrise_daily(csv_file: str) -> pd.core.frame.DataFrame:
    df = pd.read_csv(csv_file)
    df.drop(df.columns.difference(['DATE','Sunset','Sunrise']), 1, inplace=True)
    df = df.dropna()
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.astype({'Sunrise': 'int32', 'Sunset': 'int32', 'DATE':'datetime64[ns]' })
    return df

In [ ]:
def load_and_clean_sunrise_sunset_data() -> pd.core.frame.DataFrame:
    sunrise_sunset_dataframes =[]
    
    weather_csv_files = [
            "/content/2009_weather.csv",
            "/content/2010_weather.csv",
            "/content/2011_weather.csv",
            "/content/2012_weather.csv",
            "/content/2013_weather.csv",
            "/content/2014_weather.csv",
            "/content/2015_weather.csv"
        ]
    
    for csv_file in weather_csv_files:
        sunrise_sunset_dataframe = clean_sunset_sunrise_daily(csv_file)
        sunrise_sunset_dataframes.append(sunrise_sunset_dataframe)
        
    sunrise_sunset_data = pd.concat(sunrise_sunset_dataframes)
    sunrise_sunset_data['DATE'] = pd.to_datetime(sunrise_sunset_data['DATE'])
    sunrise_sunset_data = sunrise_sunset_data.astype({'Sunrise': 'int32', 'Sunset': 'int32'})
    
    return sunrise_sunset_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_sample_taxi_data()
uber_data = load_and_clean_uber_data("/content/uber_rides_sample.csv")
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()
sunrise_sunset_data = load_and_clean_sunrise_sunset_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
from sqlalchemy import Column, Date, Integer, Float, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [ ]:
class HourlyWeather(Base):
    __tablename__ = 'hourly_weathers'
    
    id = Column(Integer, primary_key=True)
    Date = Column(Date)
    HourlyPrecipitation = Column(Float)
    HourlyWindSpeed = Column(Float)

class DailyWeather(Base):
    __tablename__ = 'daily_weathers'
    
    id = Column(Integer, primary_key=True)
    Date = Column(Date)
    DailyPrecipitation = Column(Float)
    DailyAverageWindSpeed = Column(Float)

class TaxiTrip(Base):
    __tablename__ = 'taxi_trips'
    
    id = Column(Integer, primary_key=True)
    pickup_datetime = Column(Date)
    trip_distance = Column(Float)
    pickup_longitude = Column(Float)
    pickup_latitude = Column(Float)
    dropoff_longitude = Column(Float)
    dropoff_latitude = Column(Float)

class UberTrip(Base):
    __tablename__ = 'uber_trips'
    
    id = Column(Integer, primary_key=True)
    pickup_datetime = Column(Date)
    trip_distance = Column(Float)
    pickup_longitude = Column(Float)
    pickup_latitude = Column(Float)
    dropoff_longitude = Column(Float)
    dropoff_latitude = Column(Float)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    weatherId INTEGER PRIMARY KEY AUTOINCREMENT,
    Date DATE,
    HourlyPrecipitation FLOAT,
    HourlyWindSpeed FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    weatherId INTEGER PRIMARY KEY AUTOINCREMENT
    Date DATE,
    DailyPrecipitation FLOAT,
    DailyAverageWindSpeed FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips
(
    taxi_tripId INTEGER PRIMARY KEY AUTOINCREMENT,
    pickup_datetime DATE,
    distance FLOAT,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    uber_tripId INTEGER PRIMARY KEY AUTOINCREMENT,
    pickup_datetime DATE,
    distance FLOAT,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT
);
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
connection = sqlite3.connect("project.db")

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
uber_data.to_sql("uber_trips", con = connection,schema=UBER_TRIPS_SCHEMA)
taxi_data.to_sql("taxi_trips", con = connection,schema=TAXI_TRIPS_SCHEMA)
daily_weather_data.to_sql("daily_weathers", con = connection,schema=DAILY_WEATHER_SCHEMA)
hourly_weather_data.to_sql("hourly_weathers", con = connection,schema=HOURLY_WEATHER_SCHEMA)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query: str, outfile: str):
    with open(outfile, "w") as f:
        f.write(query)
        f.close()

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_1 = """
SELECT strftime ('%H',pickup_datetime) AS HOUR,
COUNT(strftime ('%H',pickup_datetime)) AS Trip_counts
FROM taxi_trips
GROUP BY HOUR
ORDER BY Trip_counts DESC
"""

In [ ]:
engine.execute(QUERY_1).fetchall()

In [ ]:
write_query_to_file(QUERY_1, "Q1.sql")

In [ ]:
QUERY_2 = '''
SELECT strftime ('%w',pickup_datetime) AS Day,
COUNT(strftime ('%w',pickup_datetime)) AS Trip_counts
FROM uber_trips
GROUP BY Day
ORDER BY Trip_counts DESC
'''

In [ ]:
engine.execute(QUERY_2).fetchall()

In [ ]:
write_query_to_file(QUERY_2, "Q2.sql")

In [ ]:
QUERY_3 = """
SELECT distance FROM(
SELECT trip_distance AS distance
FROM (SELECT pickup_datetime, trip_distance, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude from taxi_trips
WHERE pickup_datetime between '2013-07-01' AND '2013-07-31'
union all
SELECT pickup_datetime, trip_distance, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude from uber_trips)
WHERE pickup_datetime between '2013-07-01' AND '2013-07-31'
)
ORDER BY distance ASC
LIMIT 1
OFFSET (SELECT count(*)
FROM (SELECT trip_distance AS distance
FROM (SELECT pickup_datetime, trip_distance, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude from taxi_trips
WHERE pickup_datetime between '2013-07-01' AND '2013-07-31'
union all
SELECT pickup_datetime, trip_distance, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude from uber_trips)
WHERE pickup_datetime between '2013-07-01' AND '2013-07-31'
)) * 95 / 100 -1
"""

In [ ]:
engine.execute(QUERY_3).fetchall()

In [ ]:
write_query_to_file(QUERY_3, "Q3.sql")

In [ ]:
QUERY_4 = """
SELECT strftime ('%Y-%m-%d',pickup_datetime) AS DAY,
AVG(trip_distance)
FROM uber_trips
WHERE pickup_datetime between '2009-01-01' AND '2009-12-31'
GROUP BY DAY
ORDER BY COUNT(strftime ('%Y-%m-%d',pickup_datetime)) DESC
LIMIT 10
"""

In [ ]:
engine.execute(QUERY_4).fetchall()

In [ ]:
write_query_to_file(QUERY_4, "Q4.sql")

In [ ]:
QUERY_5 = """
SELECT DAY, COUNT(*) FROM
(SELECT strftime('%Y-%m-%d', pickup_datetime) as DAY from taxi_trips
UNION ALL
SELECT strftime('%Y-%m-%d', pickup_datetime) as DAY from uber_trips)
WHERE DAY IN (SELECT strftime ('%Y-%m-%d', DAY) AS DAY FROM daily_weathers
WHERE DAY between '2014-01-01' AND '2014-12-31'
ORDER BY (DailyAverageWindSpeed) DESC
LIMIT 10)
GROUP BY DAY
"""

In [ ]:
engine.execute(QUERY_5).fetchall()

In [ ]:
write_query_to_afile(QUERY_5, "Q5.sql")

In [ ]:
QUERY_6 = """
SELECT weathers.DATE, weathers.HOUR, weathers.HourlyPrecipitation, weathers.HourlyWindSpeed, trips.numbers
FROM
(
SELECT strftime ('%Y-%m-%d',Date) AS DATE, strftime ('%H',Date) AS HOUR, HourlyPrecipitation, HourlyWindSpeed
FROM hourly_weathers 
WHERE DATE BETWEEN '2012-10-22' AND '2012-11-07'
GROUP BY strftime ('%Y-%m-%d',Date), strftime ('%H', Date), HourlyPrecipitation, HourlyWindSpeed
) weathers
LEFT JOIN
(
SELECT strftime ('%Y-%m-%d',pickup_datetime) AS DATE, strftime ('%H',pickup_datetime) AS HOUR, COUNT(strftime ('%H',pickup_datetime)) as numbers FROM taxi_trips
WHERE DATE BETWEEN '2012-10-22' AND '2012-11-07'
GROUP BY strftime ('%Y-%m-%d',pickup_datetime), strftime ('%H',pickup_datetime) 
union all
SELECT strftime ('%Y-%m-%d',pickup_datetime) AS DATE, strftime ('%H',pickup_datetime) AS HOUR, COUNT(strftime ('%H',pickup_datetime)) as numbers FROM uber_trips
WHERE DATE BETWEEN '2012-10-22' AND '2012-11-07'
GROUP BY strftime ('%Y-%m-%d',pickup_datetime), strftime ('%H',pickup_datetime) 
) trips
ON trips.DATE = weathers.DATE AND trips.HOUR = weathers.HOUR
GROUP BY weathers.DATE, weathers.HOUR, weathers.HourlyPrecipitation, weathers.HourlyWindSpeed, trips.numbers
"""

In [ ]:
engine.execute(QUERY_6).fetchall()

In [ ]:
write_query_to_file(QUERY_6, "Q6.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

### Visualization 1

In [ ]:
QUERY_4_1 = """
SELECT strftime ('%H',pickup_datetime) AS HOUR,
COUNT(strftime ('%H',pickup_datetime)) AS NUMBERS
FROM taxi_trips
GROUP BY HOUR
"""

def get_data_for_visual_1():
    df = pd.read_sql(QUERY_4_1, con = engine)
    return df

dataframe = get_data_for_visual_1()

def plot_visual_1(dataframe):
    plt.bar(df['HOUR'],df['NUMBERS'])
    plt.title('The hourly distribution of numbers of yellow taxi trips')
    plt.xlabel('Hour of the day')
    plt.ylabel('Number of yellow taxi trips')
    return plt.show()

### Visualization 2

In [ ]:
QUERY_4_2 = """
SELECT month,
    total_trips,
    AVG(avr_distance) as average_distance,
    (count(distance)*sum(distance*distance) - (sum(distance)*sum(distance)))/((count(distance)-1)*(count(distance))) as variance
    FROM 
    (SELECT 
    trip_distance AS distance,
    strftime ('%m',pickup_datetime) AS month,
    count(*) AS	total_trips,
    AVG(trip_distance) AS avr_distance
    FROM 
    taxi_trips
    GROUP BY 
    strftime ('%m',pickup_datetime)
    UNION 
    SELECT 
    trip_distance AS distance,
    strftime ('%m',pickup_datetime) AS month,
    count(*) AS	total_trips,
    AVG(trip_distance) AS avr_distance
    FROM 
    uber_trips
    GROUP BY 
    strftime ('%m',pickup_datetime)
    )trips
    GROUP BY
    trips.month
"""

def get_data_for_visual_2():
    df = pd.read_sql(QUERY_4_2, con = engine)
    return df

df = get_data_for_visual_2()

ci_lower = df['average_distance'] - 1.64 *np.sqrt(df['variance'])/np.sqrt(df["total_trips"])
ci_upper = df['average_distance'] + 1.64 *np.sqrt(df['variance'])/np.sqrt(df["total_trips"])
fig, ax = plt.subplots()
x = df['month']
ax.plot(x, df['average_distance'])
ax.fill_between(x, ci_lower, ci_upper, color='b', alpha=.15)
ax.set_ylim(ymin=2.5)
ax.set_title('average distance traveled per month')
fig.autofmt_xdate(rotation=45)

### Visualization 3

In [ ]:
JFK_BOX = [-73.825248, 40.620479, -73.746971, 40.666458]
LGA_BOX = [-73.890716,40.767245,-73.854667,40.781415]
EWR_BOX = [-74.192324,40.670659,-74.153185, 40.708601]

In [ ]:
def plot_visual_3():
    query_JFK = """
    SELECT day, COUNT(day) AS Trip_counts
    FROM
    (SELECT strftime('%w', pickup_datetime) AS day, dropoff_longitude, dropoff_latitude from taxi_trips
    UNION ALL
    SELECT strftime('%w', pickup_datetime) AS day, dropoff_longitude, dropoff_latitude from uber_trips)
    WHERE dropoff_longitude BETWEEN -73.825248 AND -73.746971 AND dropoff_latitude BETWEEN 40.620479 AND 40.666458
    GROUP BY day
    ORDER BY Trip_counts
    """
    query_LGA = """
    SELECT day, COUNT(day) AS Trip_counts
    FROM
    (SELECT strftime('%w', pickup_datetime) AS day, dropoff_longitude, dropoff_latitude from taxi_trips
    UNION ALL
    SELECT strftime('%w', pickup_datetime) AS day, dropoff_longitude, dropoff_latitude from uber_trips)
    WHERE dropoff_longitude BETWEEN -73.890716 AND -73.854667 AND dropoff_latitude BETWEEN 40.767245 AND 40.781415
    GROUP BY day
    ORDER BY Trip_counts
    """

    query_EWR = """
    SELECT day, COUNT(day) AS Trip_counts
    FROM
    (SELECT strftime('%w', pickup_datetime) AS day, dropoff_longitude, dropoff_latitude from taxi_trips
    UNION ALL
    SELECT strftime('%w', pickup_datetime) AS day, dropoff_longitude, dropoff_latitude from uber_trips)
    WHERE dropoff_longitude BETWEEN -74.192324 AND -74.153185 AND dropoff_latitude BETWEEN 40.670659 AND 40.708601
    GROUP BY day
    ORDER BY Trip_counts
    """

    df_JFK = pd.read_sql(query_JFK, con = engine)
    df_LGA = pd.read_sql(query_LGA, con = engine)
    df_EWR = pd.read_sql(query_EWR, con = engine)
    
    barWidth = 0.25
    fig = plt.subplots(figsize =(12, 8))
    br1 = np.arange(len(df_JFK['day']))
    br2 = [x + barWidth for x in br1]
    br3 = [x + barWidth for x in br2]

    plt.bar(br1, df_JFK['Trip_counts'], color ='r', width = barWidth,
            edgecolor ='grey', label ='JFK')
    plt.bar(br2, df_LGA['Trip_counts'], color ='g', width = barWidth,
            edgecolor ='grey', label ='LGA')
    plt.bar(br3, df_EWR['Trip_counts'], color ='b', width = barWidth,
            edgecolor ='grey', label ='EWR')

    # Adding Xticks
    plt.xlabel('Days in Week', fontweight ='bold', fontsize = 15)
    plt.ylabel('Trip_counts', fontweight ='bold', fontsize = 15)

    plt.legend()
    plt.show()

### Visualization 4

In [ ]:
get_map = KeplerGl(height = 500)
get_map.add_data(uber_data, name = "heatmap")
get_map.add_data(taxi_data, name = "heatmap")
get_map

### Visualization 5

In [ ]:
def plot_visual_5():
    query_visual_5 = """
    SELECT tip_amount, trip_distance FROM taxi_trips
    """

    df_tip_distance = pd.read_sql(query_visual_5, con = engine)
    plt.scatter(x = df_tip_distance['tip_amount'], y = df_tip_distance['trip_distance'])
    plt.xlabel('tip_amount') 
    plt.ylabel('trip_distance') 
    plt.title('tip_amount vs. trip_distance')
    plt.show()

### Visualization 6

In [ ]:
def plot_visual_6():
    query_taxi ="""SELECT strftime('%Y-%m-%d %H', pickup_datetime) AS DATE, tip_amount
            FROM taxi_trips
            """ 
    query_weather ="""SELECT strftime('%Y-%m-%d %H', DATE) AS DATE, HourlyPrecipitation
            FROM hourly_weathers
            """

    taxi_data = pd.read_sql_query(query_taxi, con = engine)
    weather_data = pd.read_sql_query(query_weather, con = engine)
    
    taxi_data['DATE'] = pd.to_datetime(taxi_data['DATE'], format='%Y-%m-%d %H')
    weather_data['DATE'] = pd.to_datetime(weather_data['DATE'], format='%Y-%m-%d %H')

    df = pd.merge(taxi_data, weather_data, on='DATE')
    df = df[(df['HourlyPrecipitation'] > 0) & (df['tip_amount'] < 50)]
    
    # sample the data to make the scatter plot clearer
    df.plot(x="HourlyPrecipitation", y="tip_amount", kind="scatter", title="tip_amount vs. Precipitation")